In [3]:
import pandas as pd
from glob import glob as gb
import os
from fuzzywuzzy import fuzz
import wikipedia
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import process
import itertools
from fuzzywuzzy import fuzz
from collections import Counter
import random  
from datetime import datetime
import json 
import time
from dateutil.relativedelta import relativedelta

DATAPATH = "/media/ruben/Elements/PhD/data/hansard"
list_wikidata = [x for x in gb(DATAPATH + '/resources/wikidata-members/reformatted/*') if "-Q" in x]
dfm = pd.read_csv(DATAPATH + '/raw/pm/members_flattened.csv',sep='\t')
dfm = dfm.replace('9999-01-01',"na")
dfm['start'] = ["2030-01-01" if x == "9999-12-31" else x for x in dfm['start']]
dfm['end'] = ["2030-01-01" if x == "9999-12-31" else x for x in dfm['end']]
dfm['start'] = [datetime.strptime(x,'%Y-%m-%d') if x != "na" else "na" for x in dfm['start']]
dfm['end'] = [datetime.strptime(x,'%Y-%m-%d') if x != "na" else "na" for x in dfm['end']]

wikidata = pd.DataFrame()
for wikidata_ in list_wikidata:
    df_wkd = pd.read_csv(wikidata_,sep='\t')
    wikidata = wikidata.append(df_wkd)

wikidata['start'] = pd.to_datetime(wikidata['start'], format='%Y-%m-%d')
wikidata['end'] = pd.to_datetime(wikidata['end'], format='%Y-%m-%d')

In [4]:
missing_district = []
for c,membership in tqdm(dfm.iterrows()):

    # Check if party name is known. If so, continue. Everything is wonderful as it is.
    if membership['party_name'] != "na" or str(membership['start']) == "na":
        continue 

    if int(str(membership['start'])[:4]) <= 1917: #or int(str(membership['start'])[:4]) > 1936:
        continue 

    if membership['district'] not in set(wikidata['constituency']):
        missing_district.append(membership['district'])

48451it [01:08, 707.28it/s]


In [5]:
mss = set(missing_district)
mss = {k:k.replace('','') for k in mss if k != "na"}
mss = {k:v.replace('County','') for k,v in mss.items()}
mss = {k:v.replace('county','') for k,v in mss.items()}
mss = {k:v.replace('the','') for k,v in mss.items()}
mss = {k:v.replace('of','') for k,v in mss.items()}
mss = {k:v.replace('city','') for k,v in mss.items()}
mss = {k:v.replace('City','') for k,v in mss.items()}

wks = set(wikidata.constituency)

In [7]:
matches = [[k,process.extract(v,wks,limit=2)] for k,v in mss.items()]
matchdf = pd.DataFrame([[x[0],x[1][0][0],x[1][0][1],x[1][1][0],x[1][1][1]] for x in matches],columns=['mbs','wkd1','s1','wkd2','s2'])
matchdf.sort_values('s1').reset_index(drop=True).to_csv(DATAPATH + "/resources/wikidata-membership-constituency-table.csv",index=False)

In [8]:
matchdf

,mbs,wkd1,s1,wkd2,s2
0,"Southampton, Itchen",Southampton Itchen,97,Southampton,90
1,Kerry North,North Kerry,95,Kerry,90
2,Norfolk South Western,Norfolk North Western,90,South West Norfolk,87
3,Kilkenny South,South Kilkenny,95,Hackney South and Shoreditch,86
4,the City of London,City of London,90,Londonderry County,90
...,...,...,...,...,...
374,Haringey Wood Green,Wood Green,90,Hornsey and Wood Green,78
375,Fife East,East Fife,95,Fife,90
376,the County of South Yorkshire,South Ayrshire,90,Hackney South and Shoreditch,86
377,the London Borough of Newham,Galway Borough,86,City of London,86
